In [ ]:
%%capture
!pip install ctgan
from ctgan import load_demo
from ctgan import CTGANSynthesizer
import pandas as pd

import torch

# Using CTGAN

In [ ]:
data = pd.read_csv("/content/sample_data/adult")
data.head(5)

In [ ]:
cat_cols = [col for col in data.columns if col.startswith("cat")]
cat_cols

In [ ]:
ctgan = CTGANSynthesizer()

In [ ]:
ctgan.fit(data, cat_cols,epochs=100)

In [ ]:
# saving the model as a pickle object
# model_path = 'models/ctgan'
# torch.save(ctgan,model_path)

In [ ]:
#load the model

loaded_model = torch.load(model_path)

samples = loaded_model.sample(1000)
samples.head(5)

In [ ]:
# import numpy as np
# np.savetxt('generated1.txt',samples,fmt='%s', delimiter="\t")

# Using T-GAN

In [ ]:
#pip install tgan

In [ ]:
continuous_columns = [0, 2, 4, 10, 11, 12]    

In [ ]:
from tgan.model import TGANModel
tgan = TGANModel(continuous_columns,steps_per_epoch=100)
tgan.fit(data)

In [ ]:
# t_samples = tgan.sample(1000)
# t_samples.head(5)

## Model Saving

In [ ]:
model_path = 'models/tgan.pkl'
tgan.save(model_path, force=True)

[0507 15:25:25 @model.py:813] Model saved successfully.


### Once the model is saved, it can be loaded back as a TGAN instance by using the TGANModel.load method:

In [ ]:
new_tgan = TGANModel.load(model_path)
new_samples = new_tgan.sample(1000)
new_samples.head(5)

[0507 15:26:28 @collection.py:146] New collections created in tower : tf.GraphKeys.REGULARIZATION_LOSSES
[0507 15:26:28 @collection.py:165] These collections were modified but restored in : (tf.GraphKeys.SUMMARIES: 0->2)
[0507 15:26:28 @sessinit.py:87] WRN The following variables are in the checkpoint, but not found in the graph: global_step, optimize/beta1_power, optimize/beta2_power
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[0507 15:26:29 @sessinit.py:114] Restoring checkpoint from output/model/model-500 ...
INFO:tensorflow:Restoring parameters from output/model/model-500


  2%|2         |4/200[00:00<00:27, 7.11it/s]


,num_0,cat_1,num_2,cat_3,num_4,cat_5,cat_6,cat_7,cat_8,cat_9,num_10,num_11,num_12,cat_13,target
0,45.334054,Private,227881.435084,HS-grad,9.001935,Divorced,Prof-specialty,Husband,White,Male,-0.001988,-0.001669,39.998152,United-States,False
1,32.827935,Private,227284.152820,HS-grad,9.001901,Never-married,Craft-repair,Not-in-family,White,Female,-0.001991,-0.001955,39.998052,United-States,False
2,54.013332,Private,314424.941520,11th,14.807193,Married-civ-spouse,Adm-clerical,Husband,White,Male,-0.001996,-0.001729,39.998037,United-States,False
3,37.927471,Local-gov,182101.362067,Some-college,10.001594,Married-civ-spouse,Craft-repair,Husband,White,Male,-0.002000,-0.001914,39.998026,United-States,False
4,35.605879,Private,232992.981508,Bachelors,9.001954,Married-civ-spouse,Machine-op-inspct,Own-child,White,Male,-0.001997,-0.001458,36.768058,United-States,False


# SMOTE

In [ ]:
result = data['target'].astype(int)
data['target'] = result
data.head(5)

,num_0,cat_1,num_2,cat_3,num_4,cat_5,cat_6,cat_7,cat_8,cat_9,num_10,num_11,num_12,cat_13,target
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,0
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,0
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,1
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,1
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,0


In [ ]:
cnt = len(cat_cols)
rename_list = []

In [ ]:
def smote(X,y,ncat,df):

  from imblearn.over_sampling import SMOTE
  for i,cat in enumerate(ncat):
        print("Before SMOTE, counts of label '{0}': {1}\n".format(i,(y == cat).sum()))
  
  sm = SMOTE(random_state = 2)
  y_ch = y.values.ravel()
  X_res, y_res = sm.fit_sample(X,y_ch)
    
  print('After OverSampling, the shape of X: {}'.format(X_res.shape))
  print('After OverSampling, the shape of y: {}\n'.format(y_res.shape)) 
    
  for i,cat in enumerate(ncat):
      print("After SMOTE, counts of label '{0}': {1}".format(i,(y_res == cat).sum()))
        
  print("-------------------------------------------------------------------------")
          
  y_res = y_res.reshape(-1, 1) #reshaping y_train to (nrows,1)
  data_res = np.concatenate((X_res, y_res), axis = 1)
  var_names = df.columns
  names = [n for n in var_names]
  dat_res = np.savetxt('sample_smote.csv', data_res, delimiter="," ''',header = ','.join(names)''')
  data_res_df = pd.DataFrame(data_res, columns = var_names)
        
  return data_res_df.tail()

X = data.loc[:,data.columns != 'target']
y = data.loc[:,data.columns == 'target']
n_cat = data['target'].unique()
smote(X,y,n_cat,data)

Before SMOTE, counts of label '0': target    37155
dtype: int64

Before SMOTE, counts of label '1': target    11687
dtype: int64



/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


ValueError: ignored